In [42]:
import sys
import os
import pandas as pd
sys.path.append("/Users/natwat/Desktop/CPSC_Projects/InvBankAI/")
PROJECT_ROOT = "/Users/natwat/Desktop/CPSC_Projects/InvBankAI/"

In [43]:

from combineSingleDB import load_and_aggregate_sentiment, full_incremental_merge, full_merge
# Paths
sent_path = "/Users/natwat/Desktop/CPSC_Projects/InvBankAI/data/news/TSLA_sentiment_combined.csv"
price_path = "/Users/natwat/Desktop/CPSC_Projects/InvBankAI/data/features/TSLA_features.csv"  # Must be minute-level
output_path = "/Users/natwat/Desktop/CPSC_Projects/InvBankAI/data/features/TSLA_features_full.csv"

# Run full pipeline
sentiment_df = load_and_aggregate_sentiment(sent_path, resample_interval="1min")
sentiment_df.tail()


,Datetime,Prob_Pos_finbert,Prob_Neu_finbert,Prob_Neg_finbert,Prob_Pos_roberta,Prob_Neu_roberta,Prob_Neg_roberta,Prob_Pos_distilbert,Prob_Neu_distilbert,Prob_Neg_distilbert,Sentiment_Avg,Sentiment_Disagreement
8028,2025-07-12 16:33:00+00:00,2.333080e-05,0.873519,1.264573e-01,0.834175,0.164590,0.001235,0.998784,0.0,0.001216,0.610994,0.535479
8029,2025-07-12 16:34:00+00:00,2.333080e-05,0.873519,1.264573e-01,0.834175,0.164590,0.001235,0.998784,0.0,0.001216,0.610994,0.535479
8030,2025-07-12 16:35:00+00:00,2.333080e-05,0.873519,1.264573e-01,0.834175,0.164590,0.001235,0.998784,0.0,0.001216,0.610994,0.535479
8031,2025-07-12 16:36:00+00:00,2.333080e-05,0.873519,1.264573e-01,0.834175,0.164590,0.001235,0.998784,0.0,0.001216,0.610994,0.535479
8032,2025-07-12 16:37:00+00:00,6.735525e-08,1.000000,1.977994e-07,0.831219,0.166395,0.002386,0.956162,0.0,0.043838,0.595794,0.519740


In [44]:

from combineSingleDB import full_merge
# Get last timestamp in existing merged file
if os.path.exists(price_path):
    df = pd.read_csv(price_path, parse_dates=["Datetime"])
    last_ts = df["Datetime"].max()
    
else:
    last_ts = None

# Run the updated merge
full_incremental_merge(sentiment_df, price_path, output_path)

# View output
df = pd.read_csv(output_path)
df.tail()

✅ Saved merged data to /Users/natwat/Desktop/CPSC_Projects/InvBankAI/data/features/TSLA_features_full.csv


,Datetime,Close,High,Low,Open,Volume,EMA20,Prob_Pos_finbert,Prob_Neu_finbert,Prob_Neg_finbert,Prob_Pos_roberta,Prob_Neu_roberta,Prob_Neg_roberta,Prob_Pos_distilbert,Prob_Neu_distilbert,Prob_Neg_distilbert,Sentiment_Avg,Sentiment_Disagreement,Return,Label
1927,2025-07-11 19:55:00+00:00,313.721405,313.750000,313.239990,313.559998,281822.0,313.448088,0.938557,0.047027,0.014416,0.26611,0.618188,0.115702,0.035811,0.0,0.964189,0.413493,0.469072,-0.000467,0
1928,2025-07-11 19:56:00+00:00,313.575012,313.969910,313.450012,313.730011,577781.0,313.460176,0.938557,0.047027,0.014416,0.26611,0.618188,0.115702,0.035811,0.0,0.964189,0.413493,0.469072,-0.000303,0
1929,2025-07-11 19:57:00+00:00,313.480011,313.629486,313.350006,313.579987,244071.0,313.462065,0.938557,0.047027,0.014416,0.26611,0.618188,0.115702,0.035811,0.0,0.964189,0.413493,0.469072,-0.000160,0
1930,2025-07-11 19:58:00+00:00,313.429993,313.500000,313.339996,313.484985,296555.0,313.459011,0.938557,0.047027,0.014416,0.26611,0.618188,0.115702,0.035811,0.0,0.964189,0.413493,0.469072,0.000223,1
1931,2025-07-11 19:59:00+00:00,313.500000,313.570007,313.279999,313.429993,545590.0,313.462915,0.938557,0.047027,0.014416,0.26611,0.618188,0.115702,0.035811,0.0,0.964189,0.413493,0.469072,NaN,0


In [50]:
import ta

df = pd.read_csv("/Users/natwat/Desktop/CPSC_Projects/InvBankAI/data/features/TSLA_features_full.csv", parse_dates=["Datetime"])

# Add rolling indicators
df["SMA_50"] = ta.trend.sma_indicator(df["Close"], window=50)
df["RSI_14"] = ta.momentum.RSIIndicator(df["Close"], window=14).rsi()
df["MACD"] = ta.trend.macd_diff(df["Close"])
vwap = ta.volume.VolumeWeightedAveragePrice(
    high=df["High"], low=df["Low"], close=df["Close"], volume=df["Volume"]
)
df["VWAP"] = vwap.vwap  



In [51]:
df["Return_1min"] = df["Close"].pct_change()
df["Volume_Change"] = df["Volume"].pct_change()
df["Price_Diff"] = df["Close"] - df["Open"]

In [52]:
# Drop rows with NaN from rolling indicators
df.dropna(inplace=True)

# Define your input features
feature_cols = [
    "Close", "High", "Low", "Open", "Volume", "EMA20", "SMA_50", "RSI_14", "MACD", "VWAP",
    "Price_Diff", "Return_1min", "Volume_Change",
    "Prob_Pos_distilbert", "Prob_Neu_distilbert", "Prob_Neg_distilbert",
    "Prob_Pos_roberta", "Prob_Neu_roberta", "Prob_Neg_roberta",
    "Prob_Pos_finbert", "Prob_Neu_finbert", "Prob_Neg_finbert",
    "Sentiment_Avg", "Sentiment_Disagreement"
]

# Your label (used later for evaluation, not part of input state)
label_col = "Label"

In [53]:
df.to_csv("/Users/natwat/Desktop/CPSC_Projects/INVBANKAI/data/final/TSLA_ready.csv", index=False)


In [54]:
df.tail()

,Datetime,Close,High,Low,Open,Volume,EMA20,Prob_Pos_finbert,Prob_Neu_finbert,Prob_Neg_finbert,...,Sentiment_Disagreement,Return,Label,SMA_50,RSI_14,MACD,VWAP,Return_1min,Volume_Change,Price_Diff
1926,2025-07-11 19:54:00+00:00,313.559998,313.709991,313.100006,313.690002,306541.0,313.419318,0.938557,0.047027,0.014416,...,0.469072,0.000515,1,312.793815,53.503516,-0.024470,313.474271,-0.000335,0.239180,-0.130005
1927,2025-07-11 19:55:00+00:00,313.721405,313.750000,313.239990,313.559998,281822.0,313.448088,0.938557,0.047027,0.014416,...,0.469072,-0.000467,0,312.854843,55.498584,-0.019233,313.497728,0.000515,-0.080638,0.161407
1928,2025-07-11 19:56:00+00:00,313.575012,313.969910,313.450012,313.730011,577781.0,313.460176,0.938557,0.047027,0.014416,...,0.469072,-0.000303,0,312.915944,53.266193,-0.026317,313.529570,-0.000467,1.050163,-0.154999
1929,2025-07-11 19:57:00+00:00,313.480011,313.629486,313.350006,313.579987,244071.0,313.462065,0.938557,0.047027,0.014416,...,0.469072,-0.000160,0,312.968944,51.809749,-0.037413,313.541540,-0.000303,-0.577572,-0.099976
1930,2025-07-11 19:58:00+00:00,313.429993,313.500000,313.339996,313.484985,296555.0,313.459011,0.938557,0.047027,0.014416,...,0.469072,0.000223,1,313.015744,51.018785,-0.047485,313.547352,-0.000160,0.215036,-0.054993
